# Import Python libraries

In [1]:
import xlwings as xw
import pandas as pd
import numpy as np
from poes.model.utils import param_poes

In [2]:
xb = xw.Book('poes/controller/control.xlsm')

In [3]:
sheet = xb.sheets['Resumen']

In [4]:
df_poes = sheet['A2'].options(pd.DataFrame, index=False, expand='table').value
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,400.0,600.0,0.3,50.0,NaN
1,Espesor,40.00,Triangular,0.0,90.0,0.3,0.0,180.0
2,Porosida,0.16,Log Normal,0.0,0.2,0.8,0.0,0.4
3,Swi,0.51,Normal,0.4,0.2,NaN,0.0,1.0
4,Boi,2.60,Exponencial,1.0,0.2,NaN,1.0,2.0


# Exploratory data analysis

In [5]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min.',
       'Lim. Max.'],
      dtype='object')

In [6]:
df_poes.columns = df_poes.columns.str.strip()

In [7]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min.',
       'Lim. Max.'],
      dtype='object')

In [8]:
df_poes['Dist.'] = ['Triangular', 'Triangular', 'Log Normal', 'Normal', 'Exponencial']
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,400.0,600.0,0.3,50.0,NaN
1,Espesor,40.00,Triangular,0.0,90.0,0.3,0.0,180.0
2,Porosida,0.16,Log Normal,0.0,0.2,0.8,0.0,0.4
3,Swi,0.51,Normal,0.4,0.2,NaN,0.0,1.0
4,Boi,2.60,Exponencial,1.0,0.2,NaN,1.0,2.0


In [9]:
df_poes.shape

(5, 8)

In [10]:
df_poes['Loc'] = np.array([400, 0, 0, 0.4, 1])
df_poes['Scale'] = np.array([600, 90, 0.2, 0.2, 0.2])
df_poes['Sc'] = np.array([0.3, 0.3, 0.8, 0, 0.7])
df_poes['Lim. Min.'] = np.array([50, 0, 0, 0, 1])
df_poes['Lim. Max.'] = np.array([0, 180, 0.4, 1, 2])
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,400.0,600.0,0.3,50,0.0
1,Espesor,40.00,Triangular,0.0,90.0,0.3,0,180.0
2,Porosida,0.16,Log Normal,0.0,0.2,0.8,0,0.4
3,Swi,0.51,Normal,0.4,0.2,0.0,0,1.0
4,Boi,2.60,Exponencial,1.0,0.2,0.7,1,2.0


# Use Loc method from pandas to extract specific values

In [11]:
df_poes.loc[2, "Data values"]

0.16

In [12]:
df_poes.loc[3, "Variables"]

'Swi'

# Call param_poes function

In [13]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min.',
       'Lim. Max.'],
      dtype='object')

In [14]:
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,400.0,600.0,0.3,50,0.0
1,Espesor,40.00,Triangular,0.0,90.0,0.3,0,180.0
2,Porosida,0.16,Log Normal,0.0,0.2,0.8,0,0.4
3,Swi,0.51,Normal,0.4,0.2,0.0,0,1.0
4,Boi,2.60,Exponencial,1.0,0.2,0.7,1,2.0


In [16]:
param_poes(df_poes, 1, 'Dist.', 'Loc', 'Scale', 1000, 'Sc', 'Lim. Min.', 'Lim. Max.')

array([61.5388715 , 37.88317642, 26.86078914, 49.94823262, 49.09243675,
       29.79390669, 22.46724813, 65.64534434, 44.66101908, 37.37444044,
       15.48536097, 37.58690428, 60.84689281, 39.71062149, 67.69768961,
       42.86622428,  3.44554549, 60.92277731, 39.76622695, 39.61437938,
       28.93887485, 43.05655508,  8.12767878, 54.86191225, 74.50685632,
       52.44657519, 12.25861048,  8.92238149, 69.3130316 , 57.47961808,
       74.43693171, 54.60742286, 30.58501241, 48.22872458, 17.5310611 ,
       80.30054469, 59.47360769, 47.99224406, 59.02271972, 58.16681661,
       24.52556977, 55.41232451, 28.09000557, 46.43809447, 31.36175359,
       27.4287193 , 56.42857083, 43.29268268, 56.33885768, 39.03941875,
       26.0747752 , 76.49258068, 20.73405376, 28.86138313, 13.44362268,
       20.77769182, 15.98055592, 20.69187207, 13.11378165, 34.823774  ,
       32.56357661, 66.90487898, 25.11087313, 16.97528638,  8.82587768,
       52.3856525 , 55.13324937, 63.27226901, 17.57806518, 23.32

In [19]:
param_poes(df_poes, 4, 'Dist.', 'Loc', 'Scale', 1000, 'Sc', 'Lim. Min.', 'Lim. Max.')

array([1.15549443, 1.18681075, 1.08292768, 1.08811372, 1.45747962,
       1.09355517, 1.1538676 , 1.00284801, 1.0454375 , 1.00636841,
       1.15292345, 1.0054477 , 1.04249952, 1.10603881, 1.06941014,
       1.04302238, 1.1327148 , 1.00099077, 1.01369698, 1.02760293,
       1.42383735, 1.5260823 , 1.17616986, 1.41829214, 1.02024302,
       1.16669672, 1.18994389, 1.35546491, 1.01221839, 1.10155725,
       1.01176196, 1.05114793, 1.54293363, 1.18664865, 1.64463547,
       1.31907779, 1.02809712, 1.0853791 , 1.01889646, 1.07536382,
       1.35830078, 1.13360828, 1.53402994, 1.02586643, 1.07123895,
       1.02985583, 1.00383518, 1.7981135 , 1.34242481, 1.0038976 ,
       1.22261314, 1.01685318, 1.23119943, 1.04249688, 1.00215369,
       1.19345252, 1.07232137, 1.01571882, 1.15923844, 1.03233264,
       1.1119501 , 1.00858585, 1.40814717, 1.32830891, 1.57375397,
       1.08540326, 1.47455216, 1.13017715, 1.0012991 , 1.03134197,
       1.40411529, 1.2380305 , 1.06349665, 1.2198667 , 1.05274